## 1. Deploy on SageMaker

### 1.1 Build Container and push to ECR

In [ ]:
VERSION='latest'
inference_image='sagemaker/triposr'

!bash build_and_push.sh {inference_image} {VERSION}

### 1.1 Init SageMaker session

In [73]:
# !pip install boto3 sagemaker transformers
import re
import json
import os
import boto3
import sagemaker
from sagemaker import Model
boto_sess = boto3.Session()

sess = sagemaker.session.Session(boto_session=boto_sess)
role = sagemaker.get_execution_role()
# role = 'arn:aws:iam::434444145045:role/sagemaker-modelhub'
region = boto_sess.region_name

[01/17/25 04:48:21] INFO     Found credentials from IAM Role: admin_role_for_workshop           ]8;id=816231;file:///opt/conda/envs/tripo/lib/python3.12/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=687165;file:///opt/conda/envs/tripo/lib/python3.12/site-packages/botocore/credentials.py#1075\1075]8;;\

In [74]:
CONTAINER=f"{sess.account_id()}.dkr.ecr.{region}.amazonaws.com/{inference_image}:{VERSION}"
print(CONTAINER)

434444145045.dkr.ecr.us-east-1.amazonaws.com/sagemaker/triposr:latest


- make a dummy tar.gz

In [75]:
!mkdir dummy
!tar czvf dummy.tar.gz dummy/
s3_code_prefix = f"sagemaker_endpoint/dummy"
bucket = sess.default_bucket() 
code_artifact = sess.upload_data("dummy.tar.gz", bucket, s3_code_prefix)

mkdir: cannot create directory ‘dummy’: File exists


dummy/


#### （可选）设置worker数量，一个worker代表一个模型copy，默认是1，如果卡比较大可以设置多份copy

In [99]:
env = {
    "WORKERS":"2" #需要str类型
} 

In [100]:

model = Model(
    name=sagemaker.utils.name_from_base("triposr")+"-model",
    model_data=code_artifact,
    image_uri=CONTAINER,
    role=role,
    sagemaker_session=sess,
    env=env
)

# 部署模型到endpoint
endpoint_name = sagemaker.utils.name_from_base("triposr")+"-endpoint"
print(f"endpoint_name: {endpoint_name}")
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.g5.2xlarge',
    endpoint_name=endpoint_name,
)

endpoint_name: triposr-2025-01-17-05-21-25-425-endpoint


[01/17/25 05:21:25] INFO     Creating model with name: triposr-2025-01-17-05-21-25-424-model        ]8;id=155801;file:///opt/conda/envs/tripo/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=253636;file:///opt/conda/envs/tripo/lib/python3.12/site-packages/sagemaker/session.py#4094\4094]8;;\

[01/17/25 05:21:26] INFO     Creating endpoint-config with name                                     ]8;id=864087;file:///opt/conda/envs/tripo/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=871963;file:///opt/conda/envs/tripo/lib/python3.12/site-packages/sagemaker/session.py#5889\5889]8;;\
                             triposr-2025-01-17-05-21-25-425-endpoint                                              

                    INFO     Creating endpoint with name triposr-2025-01-17-05-21-25-425-endpoint   ]8;id=171175;file:///opt/conda/envs/tripo/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=433212;file:///opt/conda/envs/tripo/lib/python3.12/site-packages/sagemaker/session.py#4711\4711]8;;\

----------------------!

## Inference

In [78]:
from sagemaker import Predictor
from sagemaker import serializers, deserializers

In [101]:
predictor = Predictor(
            endpoint_name=endpoint_name,
            sagemaker_session=sess,
            serializer=serializers.JSONSerializer(),
        )

In [102]:
from PIL import Image
import base64
import io
import boto3
import os

In [103]:
#download file from s3 url
def download_file(s3_url, local_file_path):
    s3 = boto3.client('s3')
    bucket_name = s3_url.split('/')[2]
    key = '/'.join(s3_url.split('/')[3:])
    file_name = os.path.basename(key)
    local_file_path = os.path.join(local_file_path, file_name)
    s3.download_file(bucket_name, key, local_file_path)
    return local_file_path

In [104]:
#read a image file as a  base64 string
image_path = "examples/chair.png"
with open(image_path, "rb") as f:
    binary_data = f.read()
    base_64_encoded_data = base64.b64encode(binary_data)
    base64_string = base_64_encoded_data.decode("utf-8")

In [120]:

payload = {
    "image":[base64_string],
    "render":True
    }

In [121]:
response = predictor.predict(payload)

In [122]:
result = json.loads(response.decode('utf-8'))
result

{'results': [{'render_images': ['s3://sagemaker-us-east-1-434444145045/sagemaker-endpoint/meshoutput/2025_01_17_05_48_25bw22edory9s3zjnr/0_render_000.png',
    's3://sagemaker-us-east-1-434444145045/sagemaker-endpoint/meshoutput/2025_01_17_05_48_25bw22edory9s3zjnr/0_render_001.png',
    's3://sagemaker-us-east-1-434444145045/sagemaker-endpoint/meshoutput/2025_01_17_05_48_25bw22edory9s3zjnr/0_render_002.png',
    's3://sagemaker-us-east-1-434444145045/sagemaker-endpoint/meshoutput/2025_01_17_05_48_25bw22edory9s3zjnr/0_render_003.png',
    's3://sagemaker-us-east-1-434444145045/sagemaker-endpoint/meshoutput/2025_01_17_05_48_25bw22edory9s3zjnr/0_render_004.png',
    's3://sagemaker-us-east-1-434444145045/sagemaker-endpoint/meshoutput/2025_01_17_05_48_25bw22edory9s3zjnr/0_render_005.png',
    's3://sagemaker-us-east-1-434444145045/sagemaker-endpoint/meshoutput/2025_01_17_05_48_25bw22edory9s3zjnr/0_render_006.png',
    's3://sagemaker-us-east-1-434444145045/sagemaker-endpoint/meshoutput/202

In [124]:
s3path = result['results'][1]['mesh_path']

In [125]:
#create a output folder and if exists is ok
os.makedirs('output', exist_ok=True)
download_file(s3path, 'output')

'output/0_mesh.obj'

### 并发测试

In [133]:
base64_strings = [base64_string]*1

from concurrent.futures import ThreadPoolExecutor
import concurrent

def predict_single(base64_string):
    return predictor.predict({"image": [base64_string]})

# 创建线程池执行器
with ThreadPoolExecutor(max_workers=4) as executor:
    # 提交任务
    future_to_string = {executor.submit(predict_single, base64_str): base64_str 
                       for base64_str in base64_strings}
    
    # 获取结果
    responses = []
    for future in concurrent.futures.as_completed(future_to_string):
        try:
            response = future.result()
            responses.append(response)
        except Exception as e:
            print(f'Generated an exception: {e}')

In [134]:
responses

[b'{"results":[{"mesh_path":"s3://sagemaker-us-east-1-434444145045/sagemaker-endpoint/meshoutput/2025_01_17_05_52_17o8gup4a66qw0vn2w/0_mesh.obj"}]}']